In [ ]:
!pip install transformers==4.15.0

In [ ]:
import transformers
from transformers import GPT2LMHeadModel,BertTokenizer
import numpy as np

In [ ]:
# 加载字典
def load_vocab(vocab_file):
  """Loads a vocabulary file into a dictionary."""
  vocab_dicts = {}
  with open(vocab_file, "r", encoding="utf-8") as reader:
    tokens = reader.readlines()
  for index, token in enumerate(tokens):
    token = token.rstrip("\n")
    vocab_dicts[token] = index
  print('len(vocab_dicts):',len(vocab_dicts))
  return vocab_dicts

In [ ]:
def load_para(intToTok_path,tokToInt_path):
  # 文本的jieba词语字典
  dict_intToTok = np.load(intToTok_path,allow_pickle=True).item()
  dict_TokToInt = np.load(tokToInt_path,allow_pickle=True).item()
  return dict_intToTok,dict_TokToInt

In [ ]:
# 将分词的词语用berttokenizer分词，然后从字向量中遍历取得各向量并求平均
def get_gpt2_wordsweight(tokenizer,dict_intToTok,vocab_dicts,wholeEmbedding):
  wordsweight=[]
  for num,tok in dict_intToTok.items():  # 结巴分词后的词语
    tok_charList = tokenizer.tokenize(tok)   # 分成字符
    tokweight=[]
    for char in tok_charList:
      try:
        index = vocab_dicts[char]
        tokweight.append(wholeEmbedding[index]) 
        #print(charsweight[index])
      except:
        print(char+"不在字典中")
        if char == '\n':
          tokweight.append(wholeEmbedding[vocab_dicts['[SEP]']]) 
    total = [0]*768
    if tokweight==[]:
      tokweight.append(np.array([0]*768))
    for i in range(len(tokweight)):
      total += np.array(tokweight[i])
    wordsweight.append( total / len(tokweight) )
  return np.array(wordsweight)

In [ ]:
# 获取gpt2全部21128字向量
def get_wholeEmbed(model):
  embed_weight=model.transformer.wte.weight # Word Token Embeddings
  print(embed_weight.shape)
  np.save('wholebart_charsweight.npy', embed_weight.detach().numpy())
  return embed_weight.detach().numpy()

In [ ]:
dict_intToTok,dict_TokToInt = load_para(intToTok_path,tokToInt_path)

In [ ]:
tokenizer = BertTokenizer(vocab_file=vocab_path)
model_gpt2 = GPT2LMHeadModel.from_pretrained(filename)

In [ ]:
wholeEmbedding = get_wholeEmbed(model_gpt2) # 获取bart全部21128字向量
vocab_dicts = load_vocab(vocab_path)

torch.Size([21128, 768])
len(vocab_dicts): 21128


In [ ]:
wordEmbedding = get_gpt2_wordsweight(tokenizer,dict_intToTok,vocab_dicts,wholeEmbedding)

In [ ]:
wordEmbedding

array([[-0.03963371,  0.02159631, -0.01067799, ...,  0.02621134,
         0.02223622, -0.01290673],
       [ 0.0731354 ,  0.01905653, -0.03433763, ...,  0.00874878,
        -0.01870194,  0.02161469],
       [ 0.08375222,  0.03868634, -0.0608225 , ...,  0.10359563,
         0.02695078, -0.00318346],
       ...,
       [-0.01315529,  0.04295164, -0.04904548, ..., -0.00890463,
         0.01013792, -0.04075794],
       [ 0.02195235,  0.04152397, -0.0969019 , ...,  0.00263877,
        -0.01511827,  0.05970754],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])